In [ ]:
import random
import sys
import os
import datetime
import argparse
import cv2
import numpy as np
import gc
from numpy.core.fromnumeric import _trace_dispatcher
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Activation,PReLU
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import initializers
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import math

In [ ]:
df_preds = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
weightdatapath = Path("../input/weight-multi-20210919")
testdatapaht = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"

In [ ]:
height = 256
width = 256
channel = 3
batch_size = 32
epochs = 400
seed = 26
epoch = "0924"
views = ['FLAIR', 'T1w', 'T1wCE', 'T2w']

In [ ]:
def set_seed(seed=200):
    tf.random.set_seed(seed)

    # optional
    # for numpy.random
    np.random.seed(seed)
    # for built-in random
    random.seed(seed)
    # for hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(seed)

In [ ]:
class Mish(Activation):

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'

def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

get_custom_objects().update({'Mish': Mish(mish)})

In [ ]:
class Siren(Activation):

    def __init__(self, activation, **kwargs):
        super(Siren, self).__init__(activation, **kwargs)
        self.__name__ = 'Siren'

def siren(inputs):
    return 1.0 / (1.0 + tf.math.exp(-inputs))

get_custom_objects().update({'Siren': Siren(siren)})

In [ ]:
class ARelu(Activation):

    def __init__(self, activation, **kwargs):
        super(ARelu, self).__init__(activation, **kwargs)
        self.__name__ = 'ARelu'

def arelu(x, alpha=0.90, beta=2.0):
    alpha = tf.clip_by_value(alpha, clip_value_min=0.01, clip_value_max=0.99)
    beta  = 1 + tf.math.sigmoid(beta)
    return tf.nn.relu(x) * beta - tf.nn.relu(-x) * alpha

get_custom_objects().update({'ARelu': ARelu(arelu)})

In [ ]:
class Celu(Activation):

    def __init__(self, activation, **kwargs):
        super(Celu, self).__init__(activation, **kwargs)
        self.__name__ = 'Celu'

def celu(x,alpha=2.0 ):
    mask_greater = tf.cast(tf.greater_equal(x,0),tf.float32) * x
    mask_smaller = tf.cast(tf.less(x,0),tf.float32) * x
    middle = alpha * ( tf.exp(tf.divide(mask_smaller,alpha)) - 1)
    return middle + mask_greater

get_custom_objects().update({'Celu': Celu(celu)})

In [ ]:
class TanhExp(Activation):

    def __init__(self, activation, **kwargs):
        super(TanhExp, self).__init__(activation, **kwargs)
        self.__name__ = 'TanhExp'

def tanhexp(x):
  return x * tf.math.tanh(tf.math.exp(x))

get_custom_objects().update({'TanhExp': TanhExp(tanhexp)})

In [ ]:
class Silu(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(Silu, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_variable("kernel", 
                                    shape=[int(input_shape[-1]), 
                                           self.num_outputs])

  def call(self, input):
    return tf.nn.silu(input)

In [ ]:
def load_imgs(idx,view, ignore_zeros=True):
    imgs = {}
    
    save_ds = []
    dir_path = os.walk(os.path.join(
        testdatapaht, idx, view
    ))
    for path, subdirs, files in dir_path:
        for name in files:
            image_path = os.path.join(path, name) 
            pyds = pydicom.filereader.dcmread(image_path)
            slope = float(pyds.RescaleSlope)
            intercept = float(pyds.RescaleIntercept)
            img = intercept + pyds.pixel_array * slope
            img = cv2.resize(img,[height,width])
            save_ds.append(np.array(img))
    if len(save_ds) == 0:
        save_ds = np.zeros((1,256,256))
    imgs = np.array(save_ds)
    return imgs

In [ ]:
dim = (height,width)
t_imgs = np.empty((channel, *dim))
def data_generation(ID,view,is_Train=True):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)

    # Store sample
    idx = str(ID).zfill(5)
    imgs = load_imgs(idx,view, ignore_zeros=False)
    t_size = imgs.shape
    t_a = math.ceil(t_size[0] / 3)
    t_img =imgs[:t_a]
    t_imgs[0] = t_img.mean(axis=0) * 0.25
    t_img =imgs[t_a:t_size[0] - t_a]
    t_imgs[1] = t_img.mean(axis=0) * 0.4
    t_img =imgs[t_size[0] - t_a:]
    t_imgs[2] = t_img.mean(axis=0) * 0.25
    #img_ = imgs.mean(axis=0)
    img_ = t_imgs.transpose(1,2,0)
    return img_

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
def serialize_example_test(feature0):
  feature = {
      'image': _bytes_feature(feature0.tobytes())
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
for i in range(4):
    with tf.io.TFRecordWriter(str("./") + str("brain_test_" + views[i] + ".tfrec")) as writer:
        for x in df_preds["BraTS21ID"]:
            img = data_generation(x,views[i],False)
            example = serialize_example_test(
                img)
            writer.write(example)

In [ ]:
def deserialize_example(serialized_string):
    # 特徴量を記述するディクショナリを作成
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    parsed_record = tf.io.parse_single_example(serialized_string, image_feature_description)
    #image = tf.io.decode_image(parsed_record['image'], expand_animations = False, dtype=tf.float64, channels=3)
    image = tf.io.decode_raw(parsed_record['image'], tf.float64)
    image = tf.reshape(image,[height,width,channel])
    return image

In [ ]:
def argument_image_tw2_val(img):
  #img = tf.image.adjust_brightness(img, delta=3)
  #img = tf.image.adjust_contrast(img, 0.3)

  img = tf.cast(img, tf.float32) / 255.0
  return img

In [ ]:
class RegressionModel(tf.keras.Model):
    def __init__(self,**kwargs):
        super(RegressionModel, self).__init__(**kwargs)
        self.conv1 = tf.keras.layers.Conv2D(128, 5, strides=2, kernel_initializer=initializers.TruncatedNormal(), activation="TanhExp")
        self.bn1   = tf.keras.layers.BatchNormalization()
        self.rule1 = tf.keras.layers.PReLU(alpha_initializer=tf.initializers.constant(0.15))
        #self.rule1 = Silu(0)
        #self.rule1_1 = FReLU()
        #self.rule1 = tf.keras.layers.LeakyReLU(alpha=0.3)
        self.max1  = tf.keras.layers.MaxPooling2D(5)

        self.conv2 = tf.keras.layers.Conv2D(128, 5, strides=2, kernel_initializer=initializers.TruncatedNormal(), activation="TanhExp")
        self.bn2   = tf.keras.layers.BatchNormalization()
        self.rule2 = tf.keras.layers.PReLU(alpha_initializer=tf.initializers.constant(0.15))
        #self.rule2_1 = FReLU()
        #self.rule2 = Silu(0)
        #self.rule2 = tf.keras.layers.LeakyReLU(alpha=0.3)
        self.max2  = tf.keras.layers.MaxPooling2D(5)

        self.conv3 = tf.keras.layers.Conv2D(128, 5, strides=2, kernel_initializer=initializers.TruncatedNormal(), activation="TanhExp")
        self.bn3   = tf.keras.layers.BatchNormalization()
        self.rule3 = tf.keras.layers.PReLU(alpha_initializer=tf.initializers.constant(0.15))
        #self.rule3_1 = FReLU()
        #self.rule3 = Silu(0)
        #self.rule2 = tf.keras.layers.LeakyReLU(alpha=0.3)
        self.max3  = tf.keras.layers.MaxPooling2D(5)      

        self.conv4 = tf.keras.layers.Conv2D(128, 5, strides=2, kernel_initializer=initializers.TruncatedNormal(), activation="TanhExp")
        self.bn4   = tf.keras.layers.BatchNormalization()
        self.rule4 = tf.keras.layers.PReLU(alpha_initializer=tf.initializers.constant(0.15))
        #self.rule4_1 = FReLU()
        #self.rule4 = Silu(0)
        #self.rule2 = tf.keras.layers.LeakyReLU(alpha=0.3)
        self.max4  = tf.keras.layers.MaxPooling2D(5)

        self.conv0 = tf.keras.layers.Conv2D(128, 5, strides=2, kernel_initializer=initializers.TruncatedNormal(), activation="TanhExp")
        self.bn0   = tf.keras.layers.BatchNormalization()
        self.rule0 = tf.keras.layers.PReLU(alpha_initializer=tf.initializers.constant(0.15))
        #self.rule1 = Silu(0)
        #self.rule1_1 = FReLU()
        #self.rule1 = tf.keras.layers.LeakyReLU(alpha=0.3)
        self.max0  = tf.keras.layers.MaxPooling2D(5)

        para_relu = tf.keras.layers.LeakyReLU(alpha=0.4)
        para_relu2 = tf.keras.layers.LeakyReLU(alpha=0.8)
        #para_relu = Silu(0)
        self.dence256_1 = tf.keras.layers.Dense(256, kernel_initializer=initializers.TruncatedNormal(),activation="ARelu")
        self.dence256_1_1 = tf.keras.layers.Dense(256, kernel_initializer='he_normal',activation="Mish")
        self.dence128_1 = tf.keras.layers.Dense(128, kernel_initializer='he_normal',activation="swish")
        self.dence64_1 = tf.keras.layers.Dense(64, kernel_initializer='he_normal',activation="Celu")
        self.dropoup5_1 = tf.keras.layers.Dropout(0.3)
        self.dropoup4_1 = tf.keras.layers.Dropout(0.3)
        self.dropoup3_1 = tf.keras.layers.Dropout(0.2)

        self.dence256_2 = tf.keras.layers.Dense(256, kernel_initializer=initializers.TruncatedNormal(),activation="ARelu")
        self.dence256_2_1 = tf.keras.layers.Dense(256, kernel_initializer='he_normal',activation="Mish")
        self.dence128_2 = tf.keras.layers.Dense(128, kernel_initializer='he_normal',activation="swish")        
        self.dence64_2 = tf.keras.layers.Dense(64, kernel_initializer='he_normal',activation="Celu")
        self.dropoup5_2 = tf.keras.layers.Dropout(0.3)
        self.dropoup4_2 = tf.keras.layers.Dropout(0.3)
        self.dropoup3_2 = tf.keras.layers.Dropout(0.2)

        self.dence256_3 = tf.keras.layers.Dense(256, kernel_initializer=initializers.TruncatedNormal(),activation="ARelu")
        self.dence256_3_1 = tf.keras.layers.Dense(256, kernel_initializer='he_normal',activation="Mish")
        self.dence128_3 = tf.keras.layers.Dense(128, kernel_initializer='he_normal',activation="swish")
        self.dence64_3 = tf.keras.layers.Dense(64, kernel_initializer='he_normal',activation="Celu")
        self.dropoup5_3 = tf.keras.layers.Dropout(0.3)
        self.dropoup4_3 = tf.keras.layers.Dropout(0.3)
        self.dropoup3_3 = tf.keras.layers.Dropout(0.2)

        self.dence256_4 = tf.keras.layers.Dense(256, kernel_initializer=initializers.TruncatedNormal(),activation="ARelu")
        self.dence256_4_1 = tf.keras.layers.Dense(256, kernel_initializer='he_normal',activation="Mish")
        self.dence128_4 = tf.keras.layers.Dense(128, kernel_initializer='he_normal',activation="swish")
        self.dence64_4 = tf.keras.layers.Dense(64, kernel_initializer='he_normal',activation="Celu")
        self.dropoup5_4 = tf.keras.layers.Dropout(0.3)
        self.dropoup4_4 = tf.keras.layers.Dropout(0.3)
        self.dropoup3_4 = tf.keras.layers.Dropout(0.2)

        self.dence256 = tf.keras.layers.Dense(256, kernel_initializer=initializers.TruncatedNormal(),activation="Mish")
        self.dence256_X = tf.keras.layers.Dense(256, kernel_initializer=initializers.TruncatedNormal(),activation="gelu")
        self.dence128 = tf.keras.layers.Dense(128, kernel_initializer='he_normal',activation="Mish")
        self.dence128_X = tf.keras.layers.Dense(128, kernel_initializer='he_normal',activation="selu")               
        self.dence64 = tf.keras.layers.Dense(64, kernel_initializer='he_normal',activation="gelu")
        self.dence32 = tf.keras.layers.Dense(32, kernel_initializer='he_normal',activation="Siren")
        self.dropoup5 = tf.keras.layers.Dropout(0.5)
        self.dropoup4 = tf.keras.layers.Dropout(0.4)
        self.dropoup3 = tf.keras.layers.Dropout(0.3)
        self.dropoup2 = tf.keras.layers.Dropout(0.2)
        self.dropoup = tf.keras.layers.Dropout(0.1)
        self.dence1 = tf.keras.layers.Dense(1, activation='sigmoid')

        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, input_tensor, training=True):
        # forward pass: block 1 
        x1 = input_tensor[0]
        x1 = self.conv1(x1)
        x1 = self.bn1(x1)
        x1 = self.rule1(x1)
        x1 = self.max1(x1)
        x1 = self.dence256_1(x1)
        x1 = self.dropoup5_1(x1)
        x1 = self.dence256_1_1(x1)
        #x1 = self.dropoup3_1(x1)
        x1 = self.dence128_1(x1)
        #x1 = self.rule1_1(x1)
        x1 = self.dence64_1(x1)
        x1 = self.dropoup3_1(x1)

        x2 = input_tensor[1]
        x2 = self.conv2(x2)
        x2 = self.bn2(x2)
        x2 = self.rule2(x2)
        x2 = self.max2(x2)
        x2 = self.dence256_2(x2)
        x2 = self.dropoup5_2(x2)
        x2 = self.dence256_2_1(x2)
        x2 = self.dropoup3_2(x2)
        x2 = self.dence128_2(x2)
        #x2 = self.rule2_1(x2)
        x2 = self.dence64_2(x2)
        x2 = self.dropoup3_2(x2)

        x3 = input_tensor[2]
        x3 = self.conv3(x3)
        x3 = self.bn3(x3)
        x3 = self.rule3(x3)
        x3 = self.max3(x3)
        x3 = self.dence256_3(x3)
        x3 = self.dropoup5_3(x3)
        x3 = self.dence256_3_1(x3)
        x3 = self.dropoup3_3(x3)
        x3 = self.dence128_3(x3)
        #x3 = self.rule3_1(x3)
        x3 = self.dence64_3(x3)
        x3 = self.dropoup3_3(x3)

        x4 = input_tensor[3]
        x4 = self.conv4(x4)
        x4 = self.bn4(x4)
        x4 = self.rule4(x4)
        x4 = self.max4(x4)
        x4 = self.dence256_4(x4)
        x4 = self.dropoup5_4(x4)
        x4 = self.dence256_4_1(x4)
        x4 = self.dropoup3_4(x4)
        x4 = self.dence128_4(x4)
        #x4 = self.rule4_1(x4)
        x4 = self.dence64_4(x4)
        x4 = self.dropoup3_4(x4)

        x = tf.keras.layers.Concatenate()([x1,x2,x3, x4])
        x = self.dence256(x)
        #x = self.dropoup3(x)
        x = self.conv0(x)
        x = self.bn0(x)
        x = self.rule0(x)
        x = self.max0(x)
        x = self.dence128_X(x)
        x = self.flatten(x)
        #x = self.dence128(x)
        #x = self.dropoup(x)
        x = self.dence64(x)
        #x = self.dropoup2(x)
        x = self.dence32(x)
        #x = self.dropoup(x)
        return self.dence1(x) 

    def train_step(self, data):
        x ,y = data
        with tf.GradientTape() as tape:
            predictions = self(x, training=True)
            loss = self.compiled_loss(y,predictions,regularization_losses=self.losses)
            print(y,predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        self.compiled_metrics.update_state(y, predictions)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}

In [ ]:
loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#opt = tf.keras.optimizers.SGD(learning_rate=1e-5, decay=1e-6, momentum=0.9, nesterov=True)
#opt = tf.keras.optimizers.RMSprop(lr=1e-5, rho=0.9, epsilon=None, decay=0.0)
opt = tf.keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999)
f_pre=[]

In [ ]:
testset0 = tf.data.TFRecordDataset(str("./") +  str("brain_test_FLAIR.tfrec")).map(deserialize_example).map(argument_image_tw2_val)
testset1 = tf.data.TFRecordDataset(str("./") +  str("brain_test_T1w.tfrec")).map(deserialize_example).map(argument_image_tw2_val)
testset2 = tf.data.TFRecordDataset(str("./") +  str("brain_test_T1wCE.tfrec")).map(deserialize_example).map(argument_image_tw2_val)
testset3 = tf.data.TFRecordDataset(str("./") +  str("brain_test_T2w.tfrec")).map(deserialize_example).map(argument_image_tw2_val)

input_a = tf.keras.Input(shape=(height,width, channel), name="input_a")
input_b = tf.keras.Input(shape=(height,width, channel), name="input_b")
input_c = tf.keras.Input(shape=(height,width, channel), name="input_c")
input_d = tf.keras.Input(shape=(height,width, channel), name="input_d")

model = RegressionModel()
model([input_a,input_b,input_c,input_d])

model.compile(
  optimizer=opt, 
  loss=loss_func,
  metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryCrossentropy()]
  )

test_ds = tf.data.Dataset.zip(((testset0,testset1,testset2,testset3),)).batch(10)

for f in range(5): #Fold    
    t_weight = "weight-Regression-multi_fold_0" +str(f) + "-" + epoch + ".ckpt"
    model.load_weights(Path(weightdatapath,t_weight))
    f_pre.append(model.predict(test_ds, batch_size=batch_size))
    tf.keras.backend.clear_session()

In [ ]:
pre = np.array(f_pre)
finpre = pre.mean(axis=0)

subfilename = "submission.csv"
sub = pd.DataFrame(finpre, columns=['MGMT_value'])

df_preds["MGMT_value"] = sub

df_preds.to_csv(
    subfilename,
    index=False
    )